In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("trip_count_by_zone_sql").getOrCreate()

In [5]:
directory = "/Users/keon/fastcampus/data-engineering/01-spark/data"
trip_file = "fhvhv_tripdata_2020-03.csv"
zone_file = "taxi+_zone_lookup.csv"

In [6]:
trip_data = spark.read.csv(f"file:///{directory}/{trip_file}", inferSchema = True, header = True)
zone_data = spark.read.csv(f"file:///{directory}/{zone_file}", inferSchema = True, header = True)

In [7]:
trip_data.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [8]:
zone_data.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [9]:
trip_data.createOrReplaceTempView("trip_data")
zone_data.createOrReplaceTempView("zone_data")

In [10]:
spark.sql("select * from trip_data limit 5").show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [11]:
spark.sql("select * from zone_data limit 5").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [18]:
spark.sql("select borough, count(*) as trips from \
(select zone_data.Borough as borough \
from trip_data join zone_data on trip_data.PULocationID = zone_data.LocationID) \
group by borough").show()

+-------------+-------+
|      borough|  trips|
+-------------+-------+
|       Queens|2437383|
|          EWR|    362|
|      Unknown|    845|
|     Brooklyn|3735764|
|Staten Island| 178818|
|    Manhattan|4953140|
|        Bronx|2086592|
+-------------+-------+



In [19]:
spark.sql("select borough, count(*) as trips from \
(select zone_data.Borough as borough \
from trip_data join zone_data on trip_data.DOLocationID = zone_data.LocationID) \
group by borough").show()

+-------------+-------+
|      borough|  trips|
+-------------+-------+
|       Queens|2468408|
|          EWR|  65066|
|      Unknown| 387759|
|     Brooklyn|3696682|
|Staten Island| 177727|
|    Manhattan|4553776|
|        Bronx|2043486|
+-------------+-------+

